In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [75]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [76]:
# установим необходимые библиотеки

# для находления координат
!pip install geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="smy-application")

# для OneHotEncorder
!pip install category_encoders
import category_encoders as ce

# для обработки отзывов
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
nltk.downloader.download('vader_lexicon')

sent_analyzer = SentimentIntensityAnalyzer()

# для регулярных выражений
import re

In [77]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [78]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [79]:
df_train.info()

In [80]:
df_train.head(2)

In [81]:
df_test.info()

In [82]:
df_test.head(2)

In [83]:
sample_submission.head(2)

In [84]:
sample_submission.info()

In [85]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [86]:
data.info()

In [87]:
data.nunique(dropna=False)

## Создание, удаление и корректировка признаков

### Работаем с признаком hotel_address

In [88]:
# получаем из адреса город и страну, в которой находится отель

def get_city_country(info):
    info = info.strip()
    info_list = info.split()
    info_list = info_list[-2:]

    if info_list[0]=='United' and info_list[1]=='Kingdom':
        info_list[0]='London'
        info_list[1]='United Kingdom'

    return info_list[0] + ', ' + info_list[1]

data['location'] = data['hotel_address'].apply(get_city_country)

In [89]:
# посмотрим, сколько всего уникальных значений и сохраним их в список

location_list = list(data['location'].unique())
location_list

In [90]:
# удалим признак

data.drop('hotel_address', axis=1, inplace=True)

In [91]:
# закодируем признак location методом OneHoeEncoding

encoder = ce.OneHotEncoder(cols='location')
type_bin = encoder.fit_transform(data['location'])
data = pd.concat([data, type_bin], axis=1)

### Заполним пробелы в признаках lat, lng, используя данные из нового признака 'location'

In [92]:
# получим координаты для всех 6-ти локаций

coord_for_basic_location = dict()

for city in location_list:
    coord_city = dict()
    location = geolocator.geocode(city)
    coord_city['lat'] = location.latitude
    coord_city['lng'] = location.longitude
    coord_for_basic_location[city] = coord_city

print(coord_for_basic_location)


In [93]:
# заполним пропущенные координаты

def get_geolocation(*x):
    lat, lng, location, coord = x
    
    if np.isnan(lat) or np.isnan(lng):
        lat = coord[location]['lat']
        lng = coord[location]['lng']

    return lat, lng


data[['lat','lng']] = data[['lat','lng','location']].apply(lambda x: pd.Series(get_geolocation(*x, coord_for_basic_location)), axis=1)

In [94]:
# удалим признак

data.drop('location', axis=1, inplace=True)

In [95]:
data.info()

### Работа с признаком review_date

In [96]:
# преобразуем в дату
data['review_date'] = pd.to_datetime(data['review_date'])

#заберем из даты месяц
data['month'] = pd.DatetimeIndex(data['review_date']).month

In [97]:
# удалим признак

data.drop('review_date', axis=1, inplace=True)

### Работа с признаком tags

In [98]:
# получаем всё множество тегов и выведем их количество

def get_set_tags(tags):
    tag_set = set()

    for hotel_tags in tags:

        hotel_tags = hotel_tags.lstrip("['")
        hotel_tags = hotel_tags.rstrip("']")
        hotel_tags_list = hotel_tags.split("', '")

        for tag in hotel_tags_list:
            tag = tag.strip()
            tag_set.add(tag)

    return tag_set


tag_set = get_set_tags(data['tags'])
print(len(tag_set))

In [99]:
%%time
#считаем количесво каждого тега во всех отзывах и сохраняем всё в датафрейм, сортируем по убыванию количества

def get_count_tags(tags, data_tags):
    dict_tags = {}

    for tag in tags:
        cnt = 0

        for tags_str in data_tags:
            if tag in tags_str:
                cnt+=1
        
        dict_tags[tag]=cnt

    tags_data=pd.DataFrame({"tag": dict_tags.keys(), "count": dict_tags.values()})
    tags_data.sort_values(by='count', ascending=False, inplace=True, ignore_index=True)
    
    return tags_data

tags_data = get_count_tags(tag_set,data['tags'])

In [100]:
# выберем самые популярные теги и создадим на их основе новые признаки
popular_tag_list = list(tags_data[tags_data['count']>10000]['tag'])

for tag in popular_tag_list:
    tag_in_text = "' "+tag+" '"
    data['tag_'+tag] = data['tags'].apply(lambda x: 1 if tag_in_text in x else 0)

data.info()

In [101]:
# удалим признак
data.drop('tags', axis=1, inplace=True)

### Работа с отзывами

In [102]:
# добавим новый признак для оценки значений положительных и отрицательных отзывов

def get_review_value(*x):
    neg_review, pos_review = x

    values = sent_analyzer.polarity_scores(neg_review)
    neg_value = values['neg']

    values = sent_analyzer.polarity_scores(pos_review)
    pos_value = values['pos']

    return neg_value, pos_value

In [103]:
data[['negative_value', 'positive_value']] = data[['negative_review','positive_review']].apply(lambda x: pd.Series(get_review_value(*x)), axis=1)

In [104]:
# заменим текст отзывов на 0 или 1, если есть негативные или позитивные
data['negative_review'] = data['negative_review'].apply(lambda x: 0 if x=='No Negative' else 1)
data['positive_review'] = data['positive_review'].apply(lambda x: 0 if x=='No Positive' else 1)

### Работа с признаком days_since_review

In [105]:
# получим численное значение того, сколько дней прошло
def get_days_count(string):
    
    days_count = re.findall("\d+", string)
    return int(days_count[0])

In [106]:
data['days_since_review'] = data['days_since_review'].apply(get_days_count)

### Работа с признаком reviewer_nationality

In [107]:
# Посмотрим на соотношение национальностей гостей отелей
print(data['reviewer_nationality'].value_counts(normalize=True))

В основном (около ~50%) рецензентов из Великобритании. Создадим новый признак, Который будет равен 1, если рецендент из Великобритании. В остальных случаях = 0

In [108]:
data['reviewer_from_UK'] = data['reviewer_nationality'].apply(lambda x: 1 if x.strip()=='United Kingdom' else 0)

In [109]:
# удалим признак
data.drop('reviewer_nationality', axis=1, inplace=True)

### Работа с признаком hotel_name

In [110]:
# т.к. название отеля не должно влиять на оценку, удалим этот признак
data.drop('hotel_name', axis=1, inplace=True)

In [111]:
data.info()

## Отбор признаков

In [112]:
# т.к. признаков очень много, вместо тепловой карты напишем ф-ию, которая выведет те признаки, где корреляция > 0.7
#мультиколлинеарность
def get_mylticoll(data, val=0.7):
    correl = data.corr()

    index_list = list(correl.index)
    column_list = list(correl.columns)
    multicoll_list = []

    for index in index_list:

        for column in column_list:

            correl_index = round(correl.loc[index,column],2)
            if ((correl_index <-1*val) or (correl_index > val)) and index!=column:

                if multicoll_list.count([column,index])==0:
                    multicoll_list.append([index,column])

    return multicoll_list


print(get_mylticoll(data.drop(['sample'], axis=1)))

In [113]:
# addition_number_of_scoring, tag_Business_trip, negative_value
data.drop(['additional_number_of_scoring','tag_Business trip','negative_value','location_3','location_6'], axis=1, inplace=True)

In [114]:
plt.rcParams['figure.figsize'] = (25,25)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True)

In [115]:
data.info()

In [116]:
# укажем категориальные признаки
category_colums = ['negative_review',
       'positive_review',
       'lat', 'location_1', 'location_2',
       'location_4', 'location_5', 'month', 'tag_Leisure trip',
       'tag_Submitted from a mobile device', 'tag_Couple', 'tag_Double Room',
       'tag_Stayed 1 night', 'tag_Stayed 2 nights', 'tag_Twin Room',
       'tag_Solo traveler', 'tag_Stayed 3 nights', 'tag_Standard',
       'tag_Superior', 'tag_Double or Twin Room', 'tag_Deluxe', 'tag_Group',
       'tag_Family with young children', 'tag_Standard Double',
       'tag_Superior Double', 'tag_Stayed 4 nights', 'tag_Deluxe Double',
       'tag_Standard Double Room', 'tag_Superior Double Room',
       'tag_Deluxe Double Room', 'tag_Family with older children',
       'tag_King Room', 'tag_Single Room', 'tag_Queen Room',
       'tag_Stayed 5 nights', 'tag_Standard Double or Twin Room',
       'tag_Classic Double Room', 'tag_Suite',
       'tag_Superior Double or Twin Room', 'tag_2 rooms',
       'tag_Standard Twin Room', 'reviewer_from_UK']

In [126]:
# укажем числовые признаки
numeric_colums = ['average_score',
       'review_total_negative_word_counts',
       'total_number_of_reviews', 'review_total_positive_word_counts',
       'total_number_of_reviews_reviewer_has_given', 'days_since_review', 'positive_value']

In [127]:
# выберем в качестве Х данные для обучения и выберем категориальные признаки
X = data[data['sample']==1].drop(['reviewer_score'], axis = 1)  

# выберем в качестве у целевой признак
y = data[data['sample']==1]['reviewer_score'] 
y=y.astype('int')

In [132]:
# проверка значимости категориальных признаков методом хи квадрат

from sklearn.feature_selection import chi2 
plt.rcParams['figure.figsize'] = (15,10)
important_cat = pd.Series(chi2(X[category_colums], y)[0], index=category_colums)
important_cat.sort_values(inplace = True)
important_cat.plot(kind = 'barh')

In [141]:
# Оставим следующие категориальные признаки (рейтинг более 150)
result_columns = ['reviewer_score', 'sample']
result_columns.extend(important_cat[important_cat >= 150].index)

In [135]:
result_columns

In [136]:
# проверка значимости числовых признаков тестом anova

from sklearn.feature_selection import f_classif # anova

plt.rcParams['figure.figsize'] = (15,10)
important_num = pd.Series(f_classif(X[numeric_colums], y)[0], index = numeric_colums)
important_num.sort_values(inplace = True)
important_num.plot(kind = 'barh')

In [142]:
# Оставим следующие числовые признаки (рейтинг более 100)
result_columns.extend(important_num[important_num >= 100].index)
result_columns

In [144]:
# соберем все данные
data = data[result_columns]

In [146]:
data.info()

## Создание модели

In [147]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [148]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [150]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [151]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [152]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [153]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [154]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))

In [155]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [156]:
test_data.sample(10)

In [157]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [158]:
sample_submission

In [159]:
predict_submission = model.predict(test_data)

In [160]:
predict_submission

In [161]:
list(sample_submission)

In [162]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission5.csv', index=False)
sample_submission.head(10)